In [ ]:
import pandas as pd
import numpy as np
import csv
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
class ResponseSearch():
    
    def __init__(self,csv_file_path):
        self.csv_file_path = csv_file_path
        self.data = self._load_data()
        self.tokenizer = AutoTokenizer.from_pretrained("")
        self.model = AutoModel.from_pretrained("")

    def _load_data(self):
        data = []
        with open(self.csv_file_path,mode="r",encoding="utf-8") as file:
            reader = csv.DictReader(file)
            for row in reader:
                embedding = np.fromstring

        
